## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [9]:
import os
import getpass

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

tavily_key = get_api_key("TAVILY_API_KEY", "Tavily API Key: ")
if tavily_key:
    print("Tavily API key set")
else:
    print("Warning: No Tavily API key configured")

Anthropic API key set
Tavily API key set


## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [10]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [11]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [12]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [13]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
- The Agent is responsible for the overall coordination/orchestration, clarifying conversation with user, research brief management and notes, and final report generation. Importantly, the Agent determines the exact research question that the system will need to answer, which is then delegates to the Supervisor to start the research process.
- The Supervisor is responsible for research delegation, requiring careful understanding of the user question and the research required to answer. It also tracks tool call iterations and aggregates notes from the sub-researchers.
- The Researcher performs the actual research as delegated by the supervisor. It is responsible for broadly researching the topic and progressively narrowing down the search until the question has been sufficiently answered.

We don't use a single huge state for a few reasons:
- Each state have different purposes and scopes so not all information is needed for each part of the system
- In a single state, too many messages will cause noise and cause context bloat, impacting the LLM's performance
- Different states also save tokens ($$$) because not all messages are required to be passed around

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
Notebooks are great for teaching when learning about new concepts. Notebooks allow for intermixing of markdown and code cells better for teaching, allowing users to read, then implement.

Adding components directly into the notebook (not importing) allows users to easily read and learn more about the code directly. Additionally, the code can be placed over different cells, helping users to better comprehend the different components.

Alternatively, importing components directly is much easier and allows for more code/libraries to be pulled in within a few short lines. The code from these Python files is easily extensible and tested.

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

**Research Brief** Prompt Explanation

This brief translates the user’s request into a research topic that the rest of the system will investigate. There are two main purposes with this prompt: creating a research topic based on the user's messages (and clarification messages) and ensuring the research topic is well-defined so that autonomous agents can perform the research as desired.

To help with the above goals, the prompt uses a few techniques:
- Clearly definition of the LLM's goal and objective
- Sets guidelines for how the research topic should be defined, including user preferences, assumptions, and constraints
- Specifies a structured output format (`ResearchQuestion`), requiring the LLM to produce a single string output of a research brief which keeps the goal specific and focused

I selected this prompt because it defines the vision and focus for the entire research. One potential improvement would be to add a guideline, specifying the need for the audience and/or use case for the research. For example, the requirements differ significantly between a student learning a new topic and a business executive making a purchasing decision. This information would improve the focus of the research and evidence requirements.


---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [14]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [15]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [16]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [17]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [18]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [19]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [20]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [21]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [22]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [23]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [16]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [17]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research request. I understand that you're looking for evidence-based strategies to address your current sleep challenges, which include inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I will now research the most effective, scientifically-backed sleep hygiene practices and create a comprehensive, personalized sleep improvement plan that addresses your specific issues.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality by developing a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research the most effective, scientifically-backed sleep hygiene strategies and interventions that specifically address incon


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 36a0367d-2b4e-43a2-aa03-a9c1a1c5ba6c, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CXqUs7cwdNcqPBQvQLVjS'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
The primary benefit of parallel researchers is time reduction - research is performed faster and the overall deep reserach process completes quicker. Parallel researchers can also cover a broader spectrum of topics, helping to increase the overall robustness of the output report. 

However, parallel researchers adds overall system complexity, higher costs, and potentially more duplicated work. These trade-offs can be somewhat minimized, but typically the benefits outweigh these costs.

Sequential researchers will typically have more efficiency and reduced costs, as the system can review and adapat the next research topics/areas before deciding on next steps. Alternatively, sequential researchers will take longer to produce the overall report since it will not cover all research directions simultaneously. Another issue is that sequentially following a single path may miss another angle that could have been explored in parallel. This is not to say that sequential researchers are always worse! Each method (parallel vs sequential) has their own specific use cases.

Prefer parallel when:
- Speed is preferred
- Research requests can be easily decomposed into independent sub-topics
- Broader range of research across sources

Prefer sequential when:
- Budget is a concern and efficiency matters
- The research topic has dependencies where the output of one task depends on another
- Need a single research explanation vs a broad/exhaustive coverage

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
For a production wellness application, some key concerns are around information accuracy and safety. 

Any sources that the system uses should be curated and safe. A RAG system would be ideal, but could be too large to be effective. To keep the RAG system under control, keep the best RAG sources and cure a list of known, reputable website providing accurate wellness data. Lastly, a system to review or version the data or workflow would ensure the data is fresh and up-to-date.

Safety and guardrails are critical. The system should have rules to not specify medical advice (which should come from a professional only). Additionally, development and handling of PII should also be added to limit the spread during research.

Another important component is personalization. The system would need to store a user profile and manage long-term memory around goals and/or recent events. Much of this could come from explicit or implicit discussions with the application.

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [28]:
# Running a new research request
my_wellness_request = """
I want to improve my health and exercise routine. I currently:
- Walk for 15-30 minutes a day
- Exercise 1-2 times per week for 30 minutes a day (typically rowing or taking a HIIT class)

I am over 40 years old and want to ensure I am doing the best things for my health as I get older.
Current fitness level is high, with relatively high cardiovascular fitness. Am looking to maintain cardiovascular fitness while improving strength.
Walks are outdoors with some hills and I have a personal rower I use.
Willing to dedicate more time to exercise if needed.
It is difficult to run for long period on concrete due to knee issues.

Please research the best evidence-based strategies for improving my health and exercise routine as I get older and create a comprehensive plan for me.
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 25000,
        "thread_id": str(uuid.uuid4())
    }
}

async def run_custom_research(research_request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

In [29]:
# Run your research
await run_custom_research(my_wellness_request, my_config)

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your research request. I understand you're over 40 with high cardiovascular fitness looking to maintain cardio while improving strength. Your current routine includes daily 15-30 minute outdoor walks with hills and 1-2 weekly exercise sessions (rowing/HIIT). You have knee issues that limit running on concrete but are willing to dedicate more time to exercise. I will now research evidence-based strategies for optimizing health and exercise routines for people over 40, focusing on maintaining cardiovascular fitness while building strength, and create a comprehensive plan tailored to your situation.

Node: write_research_brief

Research Brief Generated:
I am over 40 years old with high cardiovascular fitness and currently exercise by walking 15-30 minutes daily outdoors on hills, plus rowing or HIIT classes 1-2 times per week for 30 minutes each. I have knee issues that prevent long-dist


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Evidence-Based Exercise Optimization for Adults Over 40: A Comprehensive Training Plan

## Overview and Current Guidelines

The American College of Sports Medicine (ACSM) and World Health Organization provide clear evidence-based guidelines for adults over 40. The ACSM recommends moderate-intensity aerobic exercise for at least 150 minutes per week or vigorous-intensity aerobic exercise for 75 minutes per week, combined with muscle-strengthening activities on at least two days per week [1]. The WHO 2020 guidelines similarly recommend 150-300 minutes of moderate-intensity, or 75-150 minutes of vigorous-intensity physical activity per week [2].

Your current routine of 105-210 minutes of weekly cardio (daily walking plus 1-2 exercise sessions) already meets or exceeds these cardiovascular recommendations. However, research shows that going beyond the minimum 150 minutes provides additional health benefits. Each 1 MET increase in cardiorespiratory fitness confers approximately 16% decrease in mortality, with the most physically active individuals demonstrating 7-8 year gains in life expectancy [3].

## Evidence-Based Exercise Frequency, Duration, and Intensity

### Cardiovascular Training
Research demonstrates that training intensity significantly impacts outcomes. For each 10% increase in exercise intensity (measured as percentage of peak oxygen uptake or peak heart rate), there is a substantial 1.0 ml/kg/min improvement in peak oxygen uptake [4]. Training intensity should be prescribed using 60-80% of heart rate reserve or 70-85% of maximal heart rate, with a minimum threshold of 60-70% of maximum heart rate for deconditioned individuals [3].

The MET-minutes concept provides practical guidance: 500-1,000 MET-minutes per week meets minimum criteria for effective exercise dosage. Examples include 60 minutes of walking at 3 mph (3.4 METs) three days per week equals 612 MET-minutes, or 30 minutes of vigorous activity like rowing (~7 METs) three days per week equals 630 MET-minutes [3].

### Strength Training Requirements
The ACSM emphasizes that "no one is ever too old to strength train" [5]. However, only 8.7% of adults over 75 participate in muscle-strengthening activities, despite the critical importance for healthy aging [5]. Adults begin losing muscle mass at age 30, with acceleration to 10% loss after age 60 and greater than 50% loss after age 80 in sedentary individuals [6].

## Strength Training Protocols for Adults Over 40

### Basic ACSM Recommendations
The foundational ACSM strength training recommendations include 8-10 exercises for major muscle groups, 1 set of each exercise, 8-12 repetitions per set, 2-3 non-consecutive training days per week, full-range movements (pain-free), and moderate speed movements (approximately 6 seconds per repetition) [7].

### Advanced Evidence-Based Protocols
The National Strength and Conditioning Association (NSCA) recommends that healthy older adults perform resistance training 2-3 days per week with 2-3 sets per major muscle group, using 8-10 exercises including shoulder press, chest press, lat pulldowns, and leg exercises. Training intensity should progress from low (<51% 1RM) to moderate (51-69% 1RM) and potentially high (70-85% 1RM) loads [6].

Research shows that older adults exhibit substantial improvements following heavy-very heavy strength training (80-84% and ≥85% of 1RM), with increases of 0.5-8.5% in maximal force per training session, resulting in an impressive average increase of 2.5% per session [8]. Despite these findings, many guidelines still recommend only moderate-intensity training, potentially limiting optimal adaptations.

### Specific Loading Parameters by Goal
For your goal of maintaining cardiovascular fitness while improving strength, the evidence supports:

**Muscular Strength Development:**
- Load: 70-85% 1RM for intermediate trainees
- Volume: 3-6 sets of 1-12 repetitions
- Rest periods: 2-3 minutes between exercises [7]

**Muscular Power (functional improvement):**
- Load: 40-60% 1RM 
- Volume: 1-3 sets of 3-6 repetitions
- Rest periods: 2-3 minutes [6]

## Knee-Friendly Strength Training Strategies

### Exercise Selection Considerations
Research on knee-friendly training reveals important biomechanical considerations. Deep squats and closed kinetic chain exercises requiring knee flexion over 90° increase compression between the patella and femur and should be prescribed with caution. Most open kinetic chain exercises like knee extension in the last 30° increase patellofemoral joint load and should be avoided [9].

### Recommended Knee-Friendly Exercises
Squats remain one of the best knee strengthening exercises after 40, targeting quadriceps while supporting knee flexion and improving joint stability. Start with proper form, perform 10-15 repetitions, and build strength gradually. With consistent effort, stronger knees may be noticed within 3-6 weeks [10].

Additional knee-friendly exercises include:
- Leg press (controlled range of motion)
- Step-ups (moderate height)
- Wall sits
- Glute bridges
- Lateral band walks
- Calf raises

## Weekly Routine Structure

### Comprehensive Weekly Plan
Based on the evidence, here's an optimized weekly structure that balances cardiovascular maintenance with strength gains:

**Monday: Strength Training (Upper Body Focus)**
- 5-10 minute warm-up (light rowing or walking)
- Chest press: 3 sets x 8-12 reps
- Lat pulldown: 3 sets x 8-12 reps
- Shoulder press: 3 sets x 8-12 reps
- Row variations: 3 sets x 8-12 reps
- Core work: 2 sets
- 5-8 minute cool-down

**Tuesday: Cardio + Walking**
- 30-45 minutes outdoor hill walking (maintain current routine)
- Include some intervals at higher intensity (70-85% max HR)

**Wednesday: Strength Training (Lower Body Focus)**
- 5-10 minute warm-up
- Leg press: 3 sets x 10-15 reps
- Squats (to comfortable depth): 3 sets x 10-15 reps  
- Glute bridges: 3 sets x 12-15 reps
- Calf raises: 3 sets x 15-20 reps
- Core and balance work: 10 minutes
- Cool-down

**Thursday: Rowing or HIIT**
- Maintain current 30-minute sessions
- Focus on 60-80% heart rate reserve intensity

**Friday: Strength Training (Full Body or Functional)**
- 5-10 minute warm-up
- Functional movements with moderate resistance
- Step-ups: 3 sets x 10 each leg
- Modified deadlifts (trap bar if available): 3 sets x 8-12 reps
- Push-up variations: 3 sets x 8-15 reps
- Balance and mobility work
- Cool-down

**Weekend: Active Recovery + Walking**
- Continue daily 15-30 minute walks
- Add mobility work, stretching, or recreational activities

### Session Structure
Each exercise session should follow evidence-based structure: 10-minute warm-up (facilitating transition from rest to training and reducing cardiovascular complications), 30-60 minute conditioning phase, and 5-8 minute cool-down (preventing post-exercise hypotension and allowing gradual recovery) [3].

## Modifications to Current Routine

### Walking Enhancements
Maintain your excellent daily walking habit while adding structured intervals. Research shows that varying intensity provides superior cardiovascular adaptations. Consider adding 2-3 minutes of brisk walking (70-80% max heart rate) followed by 2-3 minutes of recovery pace, repeated 3-4 times during your walk.

### Rowing Optimization  
Your rowing sessions are excellent for cardiovascular fitness and provide some upper body strengthening. To maximize benefits, structure rowing sessions with defined intensity zones: 60-70% max heart rate for base building, 70-80% for tempo work, and short intervals at 80-85% for peak fitness maintenance.

### HIIT Class Integration
Continue your HIIT classes but ensure they complement rather than compete with strength training. Schedule HIIT sessions at least 24 hours before or after intense strength sessions to optimize recovery and adaptation.

## Recovery and Mobility Considerations

### Age-Related Recovery Needs
Adults over 40 require enhanced attention to recovery strategies. Research indicates that training repetitions to failure has not been shown to promote additional physiological adaptations in older adults and may increase injury risk [6]. Focus on progressive overload through increased load or volume rather than training to failure.

### Recovery Protocols
- Allow 48-72 hours between training the same muscle groups
- Include active recovery days with light movement
- Prioritize sleep (7-9 hours) for optimal adaptation
- Consider incorporating mobility work daily for 10-15 minutes
- Monitor subjective recovery using tools like rating of perceived exertion (RPE)

### Balance Training Integration
For adults over 40, balance training becomes increasingly important. The evidence recommends one or two sets of four to 10 different balance exercises at least 3 days per week. This can be integrated into strength training sessions or performed during active recovery periods [11].

## Long-Term Health Benefits and Injury Prevention

### Cardiovascular Benefits
Maintaining and improving cardiovascular fitness provides profound long-term benefits. Research demonstrates that individuals with below-average cardiorespiratory fitness face greater cardiovascular disease risk than comparable levels of physical inactivity. The most active individuals show 7-8 year gains in life expectancy [3].

### Strength Training Benefits
Regular strength training after 40 provides multiple benefits:
- Maintenance of muscle mass and bone density
- Improved functional capacity for daily activities
- Enhanced metabolic health
- Reduced risk of falls and injuries
- Better management of chronic conditions

### Injury Prevention Strategies
The research reveals important safety considerations:
- Most exercise-related cardiovascular events (72%) occur during warm-up or cool-down phases, emphasizing the importance of proper session structure [3]
- Progressive loading prevents overuse injuries
- Proper exercise selection respects joint limitations (knee issues)
- Task-specific training may provide greater functional benefits than traditional machine-based training [12]

### Monitoring and Progression
Implement a systematic approach to monitoring progress:
- Track strength gains through load progression
- Monitor cardiovascular fitness through heart rate recovery and perceived exertion
- Assess functional improvements through activities of daily living
- Regular reassessment of training loads and recovery needs

The evidence strongly supports your current high level of cardiovascular fitness while highlighting the critical importance of adding systematic strength training. Your knee limitations can be successfully managed through appropriate exercise selection and progression. The proposed plan provides a evidence-based framework for optimizing health and fitness as you age, with flexibility to adjust based on individual response and preferences.

### Sources

[1] ACSM Physical Activity Guidelines: https://acsm.org/education-resources/trending-topics-resources/physical-activity-guidelines/

[2] World Health Organization 2020 guidelines on physical activity and sedentary behavior: https://pmc.ncbi.nlm.nih.gov/articles/PMC7719906/

[3] Physical activity, cardiorespiratory fitness, minimum and goal recommendations: https://pmc.ncbi.nlm.nih.gov/articles/PMC9586849/

[4] Exercise intensity prescription in cardiovascular rehabilitation: https://www.frontiersin.org/journals/cardiovascular-medicine/articles/10.3389/fcvm.2024.1380639/full

[5] Working with Older Adults? Don't Skimp on Strength Training: https://acsm.org/older-adults-strength-training/

[6] Resistance Training for Older Adults: New NSCA Position Stand: https://www.unm.edu/~lkravitz/Article%20folder/NSCApositionstand.html

[7] Resistance Training for Health and Fitness: https://www.prescriptiontogetactive.com/static/pdfs/resistance-training-ACSM.pdf

[8] Heavy Strength Training in Older Adults: https://pmc.ncbi.nlm.nih.gov/articles/PMC12003923/

[9] Knee Movement and Exercise Guidelines: https://www.nsca.com/education/articles/kinetic-select/knee-movement-and-exercise-guidelines/?srsltid=AfmBOoq-XE7yc9P5aAUoLVKYSuuqK2G8EauQrX7K2_hkpfj2Gr73R2tg

[10] Knee Strengthening Routine for Adults Over 40: https://www.physiomed.ca/knee-strengthening-routine-for-adults-over-40/

[11] Exercise recommendations for older adults living with chronic conditions: https://pmc.ncbi.nlm.nih.gov/articles/PMC12778376/

[12] Task-specific resistance training adaptations in older adults: https://www.frontiersin.org/journals/aging/articles/10.3389/fragi.2024.1335534/full


Research workflow completed!
